In [1]:
import json
import os
import pandas as pd
from collections import defaultdict
import numpy 
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore') 

In [3]:
pd.set_option('display.max_columns', None) # view all columns

In [4]:
# floats show comma separators by default
pd.options.display.float_format = '{:,}'.format

In [5]:
def new_name(clus):
    if clus == "FL_3_30":
        return "Chatham Walk"
    elif clus == "FL_9_8":
        return "Ribbon Grass"
    elif clus == "FL_3_10":
        return "Belmont Glen"
    else:
        return clus

In [6]:
# go into data directory
cur_dir = os.getcwd()
data_dir = r"T:\Single Family\SFR\02. DATABASE\JSON\B2R JSON Clusters\Florida"

# import data from data file
os.chdir(data_dir)

dir_files = os.listdir()

# get state data
dir_files = [file for file in dir_files if "site_plan.json" in file]
dir_files

# focus B2R Comminity names
passed = ['FL_9_8', 'FL_3_30', 'FL_3_10']

# example input file name: 'FL_3_10__B2R_cluster_site_plan.json'
# get those of interest in passed list
files = [ file for file in dir_files if file.split('__')[0] in passed]

In [7]:
state_file_name = files[-1]

In [8]:
for state_file_name in files:
    
    ######################################
    # unpack json file

    clus = state_file_name.split('__')[0]
    print(clus)

    os.chdir(data_dir) # go back to data dir
    
    # read file
    with open(state_file_name, 'r') as myfile:
        data = myfile.read()

    # parse file
    obj = json.loads(data)
    df = pd.DataFrame(obj)

    ######################################
    # reorganize df for easier analysis
    
    bins = [0,1600, 10000]

    df['sqft_bin'] = pd.cut(df['SQFT'], bins)
    df['Name'] = df['Name'].str.split('th').str[0].tolist()

    # focus columns
    tdf = pd.DataFrame(list(zip(df['Name'], df['sqft_bin'], df['Rent'], df['DateOfObservation'])),
                           columns = ['Characteristics', 'Sqft Bin', 'Rent', 'DateOfObservation'])
    # un-nest lists
    compdf = tdf.explode(['Rent', 'DateOfObservation']) 

    # 1st half of year
    first_months = [i for i in range(1, 7)]
    compdf1st = compdf[pd.to_datetime(compdf['DateOfObservation']).dt.month.isin(first_months)]

    # 2nd half of year
    last_months = [i for i in range(7, 13)]
    compdf2nd = compdf[pd.to_datetime(compdf['DateOfObservation']).dt.month.isin(last_months)]
    compdf2nd

    ########################################################
    # 1st half of year df
    
    yoy1st = pd.DataFrame(compdf1st.groupby(['Characteristics', 'Sqft Bin', pd.PeriodIndex(compdf1st['DateOfObservation'], freq="Y")], as_index = True)['Rent'].mean())
    yoy1st['Percent Diff %'] = yoy1st.pct_change()
    yoy1st.insert(0, 'Half of Year', 1)
    yoy1st = yoy1st.reset_index()


    yoy1st['Sqft Bin'] = [' <1600 sqft' if p == pd.Interval(0.0, 1600.0, closed='right') else ' >1600 sqft' for p in yoy1st['Sqft Bin'].tolist()]
    yoy1st['Home Style'] = yoy1st['Characteristics'] + yoy1st['Sqft Bin']

    # ----------------
    # ignore perc diff from different prev home style
    char_list = yoy1st['Home Style'].tolist()
    date_list = yoy1st['DateOfObservation'].tolist()
    new = yoy1st['Percent Diff %'].tolist()
    for i in range(1, len(char_list)):
        if char_list[i] != char_list[i-1]: # if not same home style
            new[i] = None
        if date_list[i] != date_list[i-1] + 1 : # if not consecutive years
            new[i] = None
    yoy1st['Percent Diff %'] = new
    # ----------------

    chars = yoy1st['Home Style'] 
    yoy1st = yoy1st.drop(['Characteristics', 'Sqft Bin', 'Home Style'], axis = 1)
    yoy1st.insert(0, 'Home Style', chars)

    ########################################################
    # 2nd half of year df

    yoy2nd = pd.DataFrame(compdf2nd.groupby(['Characteristics', 'Sqft Bin', pd.PeriodIndex(compdf2nd['DateOfObservation'], freq="Y")], as_index = True)['Rent'].mean())
    yoy2nd['Percent Diff %'] = yoy2nd.pct_change()
    yoy2nd.insert(0, 'Half of Year', 2)
    yoy2nd = yoy2nd.reset_index()
    yoy2nd

    yoy2nd['Sqft Bin'] = [' <1600 sqft' if p == pd.Interval(0.0, 1600.0, closed='right') else ' >1600 sqft' for p in yoy2nd['Sqft Bin'].tolist()]
    yoy2nd['Home Style'] = yoy2nd['Characteristics'] + yoy2nd['Sqft Bin']

    # ----------------
    # ignore perc diff from different prev home style
    char_list = yoy2nd['Home Style'].tolist()
    date_list = yoy2nd['DateOfObservation'].tolist()
    new = yoy2nd['Percent Diff %'].tolist()
    for i in range(1, len(char_list)):
        if char_list[i] != char_list[i-1]: # if not same home style
            new[i] = None
        if date_list[i] != date_list[i-1] + 1 : # if not consecutive years
            new[i] = None
    yoy2nd['Percent Diff %'] = new
    # ----------------

    chars = yoy2nd['Home Style'] 
    yoy2nd = yoy2nd.drop(['Characteristics', 'Sqft Bin', 'Home Style'], axis = 1)
    yoy2nd.insert(0, 'Home Style', chars)

    ########################################################
    # Combine Halves

    fin = pd.concat([yoy1st, yoy2nd])
    fin['DateOfObservation']
    fin.reset_index(drop=True, inplace=True)

    fin['E2023 Rent'] = None
    # estimate rental rates for 2023
    for index, row in fin.iterrows():
        perc_diff = row['Percent Diff %']
        rent = row['Rent']
        if (str(row['DateOfObservation']) == '2022') & ~(np.isnan(perc_diff)):
            fin.at[index, 'E2023 Rent'] = round(rent + (rent * perc_diff), 2)
    #fin['Percent Diff %'] = fin['Percent Diff %'] * 100 # format

    # rename and reformat
    fin['Date'] = fin['DateOfObservation'].astype(str) + '-' + fin['Half of Year'].astype(str) + 'H'  
    fin = fin.drop(['DateOfObservation', 'Half of Year'], axis = 1)

    # break by rent and future rent columns and then concatenate them
    f1 = fin.pivot_table(['Rent', 'Percent Diff %'], ['Home Style'], ['Date'])
    f2 = fin.pivot_table(['E2023 Rent'], ['Home Style'], ['Date'])
    
    # merge halves tables
    final = pd.concat([f1,f2], axis = 1)
    #final = final.round(2) 
    # format
    final['Percent Diff %'] = final['Percent Diff %'].round(4)
    final['Rent'] = final['Rent'].round(2)
    final = final.reindex(sorted(final.columns), axis=1)
    final = final.rename({'Percent Diff %': 'Rent % Growth'}, axis=1)  
    
    this_dir = r"C:\Users\jnunez\Documents\Projects\SFR B2R Community Clustering\Cluster Analysis"
    os.chdir(this_dir)
    clus = new_name(clus)
    #yoy1st.to_csv(clus + "_YOY_1stHalf.csv", header=True, index=False)
    #yoy2nd.to_csv(clus + "_YOY_2ndHalf.csv", header=True, index=False)
    final.to_csv(clus + "_YOY.csv", header=True, index=True)
    final.to_excel(clus + "_YOY.csv", header=True, index=True)

FL_3_10
FL_3_30
FL_9_8


In [12]:
final

E2023 Rent Rent % Growth             Rent           \
Date                      2022-2H       2022-2H 2023-1H  2021-2H  2022-1H   
Home Style                                                                  
3 Bd 2 Ba <1600 sqft     2,536.54        0.1416  0.0415 1,946.25 2,110.02   
3 Bd 2 Ba >1600 sqft     2,446.16        0.1059     NaN  2,000.0 2,222.81   
4 Bd 2 Ba >1600 sqft     2,318.57        0.0203     NaN 2,227.26 2,317.43   
4 Bd 2.5 Ba >1600 sqft        NaN           NaN     NaN      NaN 2,403.75   
4 Bd 3 Ba >1600 sqft     2,961.11        0.1235     NaN 2,346.07 2,396.36   
5 Bd 3.5 Ba >1600 sqft        NaN           NaN     NaN  2,580.0 2,743.44   

                                          
Date                    2022-2H  2023-1H  
Home Style                                
3 Bd 2 Ba <1600 sqft   2,221.88 2,197.69  
3 Bd 2 Ba >1600 sqft   2,211.86      NaN  
4 Bd 2 Ba >1600 sqft   2,272.46      NaN  
4 Bd 2.5 Ba >1600 sqft 2,422.86      NaN  
4 Bd 3 Ba >1600 sqft   2,635.71      NaN  
5 Bd 3.5 Ba >1600 sqft      NaN      NaN

# Ignore

In [ ]:
stop

In [ ]:
df['Rent % Growth'] = df['Rent % Growth'].apply(lambda x: "${:.1f}k".format((x/1000)))

In [10]:
final.columns

MultiIndex([(   'E2023 Rent', '2022-2H'),
            ('Rent % Growth', '2022-2H'),
            ('Rent % Growth', '2023-1H'),
            (         'Rent', '2021-2H'),
            (         'Rent', '2022-1H'),
            (         'Rent', '2022-2H'),
            (         'Rent', '2023-1H')],
           names=[None, 'Date'])

In [11]:
final[].columns

SyntaxError: invalid syntax (3896579830.py, line 1)

In [ ]:
final.columns = [(    'E2023 Rent', '2022-2H'),
            ('Percent Diff %', 'k'),
            ('Percent Diff %', 'k'),
            (          'Rent', '2021-2H'),
            (          'Rent', '2022-1H'),
            (          'Rent', '2022-2H'),
            (          'Rent', '2023-1H')]

In [ ]:
final

In [ ]:
final.rename({'Percent Diff %': 'Rent % Growth'}, axis=1)  # new method

In [13]:
3177 - 2501

676

In [ ]:
fin.pivot_table([ 'Rent'], ['Home Style'], ['Date'])


In [ ]:
fin.pivot_table('Rent', ['Home Style'], 'Date')


In [ ]:
fin.pivot_table([ 'Percent Diff %', 'Rent'], ['Home Style'], 'Date')


In [ ]:
# good
yoy1st = pd.DataFrame(compdf1st.groupby(pd.PeriodIndex(compdf1st['DateOfObservation'], freq="Y"), as_index = True)['Rent'].mean())
yoy1st['Percent Diff %'] = yoy1st.pct_change()
yoy1st.insert(0, 'Half of Year', 1)
yoy1st = yoy1st.reset_index()
yoy1st.insert(0, 'Characteristics', 'la')
yoy1st

In [ ]:
compdf = tdf.explode(['Rent', 'DateOfObservation'])

In [ ]:
compdf.DateOfObservation.min()

In [ ]:
compdf['DateOfObservation'] = pd.to_datetime(compdf['DateOfObservation'])

In [ ]:
compdf.resample('6M', on='DateOfObservation').mean()

In [ ]:
compdf.groupby(compdf['DateOfObservation'].dt.to_period('Q')).mean()

In [ ]:
compdf.groupby(compdf['DateOfObservation'].dt.to_period('2Y')).mean()

In [ ]:

pd.DataFrame(compdf.resample('6M', on='DateOfObservation')['Rent'].mean())
